# Suncal API User Manual: Optimizing Calibration Intervals

Calibration Interval calculations in suncal are based on NCSLI Recommended Pracitce 1 (RP-1). Methods A3 and S2 are implemented in the `TestInterval` and `BinomialInterval` classes, respectively. The other methods described in RP-1 were not implemented because they are "not recommended but remain documented in this RP to discourage its “reinvention” and maintain awareness of the drawbacks of similar methods."

In [1]:
import numpy as np
import matplotlib.pyplot as plt

## Method A3 - Test Interval Method

Use Method A3 if all historical calibrations were done at nearly the same interval. It calculates a new interval based on observed reliability of all calibrations done under the same interval.


In [2]:
from suncal.intervals import A3Params, a3_testinterval

The `a3_testinterval` method uses an `A3Params` object which stores the number of in-tolerance calibrations, total number of calibrations, the assigned interval of those calibrations, and the target reliability.

Below, y is a list of pass/fail (1/0) values on DUTs of the same category calibrated with the same interval of 365 days.
(Data from 2019 NCSLI Symposium Tutorial on Intervals).

The calculation is run and result with report attribute is returned just like other suncal model types. For this set of data, the interval recommended by the A3 calculation is 231 days.

In [3]:
y = np.array([1,1,0,1,0,1,1,1,1,0,1,1,1,1,1,0,1,1,1,0,])  # Pass/fail values
params = A3Params(intol=np.count_nonzero(y), n=len(y), I0=365, target=.95)
result = a3_testinterval(params)  # Existing interval I0=365 days.
result.interval

230.84604619402594

In [4]:
result.report.summary()

Parameter,Value
Suggested Interval,231
Calculated Interval,231
Current Interval Rejection Confidence,99.49%
True reliability range,65.16% - 82.86%
Observed Reliability,75.00% (15 / 20)
Number of calibrations used,20
Rejected calibrations (wrong interval),0


If the data takes the form of calibration dates and pass/fail status for each DUT, use the `A3Params.from_assets` classmethod to set up the calculation.
Using this model, multiple assets of the same category may be pooled  to provide more statistics for a higher confidence in the resulting interval.
Suncal will inspect the dates and include only those calibrations whose interval was close to the assigned interval I0. Some may be discarded if the actual interval was too long or short for not having reliability representative of the assigned interval length.

The last line in output table lists how many calibrations were discarded.

In [5]:
y = np.array([1,1,0,1,0,1,1,1,1,0,1,1,1,1,1,0,1,1,1,0,])  # Pass/fail values
params = A3Params.from_assets([
    {'startdates': np.arange(0, 365*(len(y)), step=365),
     'enddates': np.arange(365, 365*(len(y)+1), step=365),   # Just make all cal intervals exactly 365
     'passfail': y}])
a3_testinterval(params)

|Parameter                               | Value            |
|---------------------------------------|-----------------|
|Suggested Interval                      | 266         |
|Calculated Interval                     | 266         |
|Current Interval Rejection Confidence   | 91.37%           |
|True reliability range                  | 65.16% - 82.86%  |
|Observed Reliability                    | 75.00% (15 / 20) |
|Number of calibrations used             | 20               |
|Rejected calibrations (wrong interval)  | 0                |

## Method S2 - Binomial Method

The S2 method uses the observed measurement reliability as a function of time between calibrations, fit to several different reliability models, to determine the best interval. Use this method if historical calibrations have been made at many different intervals.

Data may be entered as time-since-calibraiton vs. reliability using `S2Params()` or as calibration time vs. pass/fail for individual assets using `S2Params.from_assets()`.

Running `s2_binom_interval()` will compute all the reliability models, assign a figure of merit to each, and return the results object, containing the interval resulting from the best model.

Working the example from Table D-1 in RP-1, which lists weeks between calibrations, number of calibrations in each interval, and observed reliability in that interval:

In [6]:
from suncal.intervals import S2Params, s2_binom_interval

In [7]:
# Reliability data from Table D-1 in RP1
ti = [4,7,10,13,21,28,40,48]           # Weeks between calibrations
ni = np.array([4,6,14,13,22,49,18,6])  # Number of calibrations in each interval of ti
ri = [1.0, .83333, .6429, .6154, .5455, .4082, .5000, .3333]    # Observed measurement reliability
params = S2Params(target=.75, ti=ti, ni=ni, ri=ri)
result = s2_binom_interval(params)
result.interval

7.0

In [8]:
result

## Best Fit Reliability Model


|Interval   | Model        | Rejection Confidence   | 95.0% Confidence Interval Range  |
|----------|------------|----------------------|---------------------------------|
|7.0        | Random Walk  | 7.8%                   | 1.8 - 11.1                       |


![IMG0][]



## All Reliability Models


|Reliability Model   | Interval   | Rejection Confidence   | F-Test    | Figure of Merit  |
|-------------------|----------|----------------------|---------|-----------------|
|Random Walk         | 7          | 7.81%                  | True      | 199.89           |
|Mixed Exponential   | 7          | 11.65%                 | True      | 128.24           |
|Log Normal          | 7          | 12.10%                 | True      | 119.88           |
|Restricted Walk     | 7          | 14.48%                 | True      | 109.05           |
|Weibull             | 6          | 16.11%                 | True      | 87.45            |
|Exponential         | 10         | 24.09%                 | True      | 21.73            |
|Mortality Drift     | 10         | 34.52%                 | True      | 15.17            |
|Warranty            | 0          | 39.22%                 | True      | 13.61            |
|Drift               | 0          | 51.80%                 | True      | 10.29            |
|Modified Gamma      | 17         | 99.13%                 | False     | 2.43             |


![IMG1][]



## Binned reliability data


|Range     | Reliability   | Number of measurements  |
|---------|-------------|------------------------|
|0 - 4     | 1.000         | 4                       |
|4 - 7     | 0.833         | 6                       |
|7 - 10    | 0.643         | 14                      |
|10 - 13   | 0.615         | 13                      |
|13 - 21   | 0.545         | 22                      |
|21 - 28   | 0.408         | 49                      |
|28 - 40   | 0.500         | 18                      |
|40 - 48   | 0.333         | 6                       |




[IMG0]: 
[IMG1]: 

To see the results from all the fit models that were evaluated:

In [9]:
result.report.allmodels()

Reliability Model,Interval,Rejection Confidence,F-Test,Figure of Merit
Random Walk,7,7.81%,True,199.89
Mixed Exponential,7,11.65%,True,128.24
Log Normal,7,12.10%,True,119.88
Restricted Walk,7,14.48%,True,109.05
Weibull,6,16.11%,True,87.45
Exponential,10,24.09%,True,21.73
Mortality Drift,10,34.52%,True,15.17
Warranty,0,39.22%,True,13.61
Drift,0,51.80%,True,10.29
Modified Gamma,17,99.13%,False,2.43


In [10]:
result.report.plot.allmodels();

### Data as individual calibrations

If the data is in terms of calibration date and pass/fail, rather than summarized reliability vs interval values, use the `S2Params.from_assets` classmethod. The data will be converted into time-betwee-calibrations vs observed measurement reliability by binning into ten intervals (by default).

In [11]:
# Make up some data based on RP-1 table D-1. Results won't be the same because RP1 used hand-selected bin edges.
enddate = [0]
passfail = [1]

def addit(passes, total, weeks):
    for i in range(total):
        enddate.append(enddate[-1] + np.random.uniform(*weeks))
    passfail.extend([1]*passes + [0]*(total-passes))
    
addit(4, 4, (2, 4))
addit(5, 6, (5, 7))
addit(9, 14, (8, 10))
addit(8, 13, (11, 13))
addit(12, 22, (19, 21))
addit(20, 49, (26, 28))
addit(9, 18, (37, 40))
addit(2, 6, (48, 51))

In [12]:
# Calculate the interval
params = S2Params.from_assets([{
    'enddates': enddate,
    'passfail': passfail}])
result = s2_binom_interval(params)
result.report.allmodels()

Reliability Model,Interval,Rejection Confidence,F-Test,Figure of Merit
Random Walk,3,0.66%,True,9976.88
Mixed Exponential,1,0.93%,True,4583.82
Log Normal,2,0.90%,True,4283.51
Restricted Walk,3,1.98%,True,3798.54
Exponential,2,3.83%,True,765.96
Mortality Drift,2,7.66%,True,382.46
Weibull,0,100.00%,False,33.10
Warranty,0,100.00%,False,28.38
Drift,0,100.00%,False,27.85
Modified Gamma,10,98.83%,False,2.77


## Variables Method

Based on Castrup "Calibration Intervals from Variables Data", 2005 NCSLI Workshop & Symposium, Washington D.C, and "Establishment and Adjustment of Calibration Intervals", NASA Measurement Quality Assurance Handbook - Annex 5 (2010). NASA-HDBK-8739.19-5.

The `variables_reliability_target` and `variables_uncertainty_target` methods implement the methods described in Castrup's paper: the Reliability Target method and the Uncertainty Target method. Both methods operate on data in the format of deviation-from-prior-calibration versus time-since-last-calibration.

Data is entered into a `VariablesData` object or using `VariablesData.from_assets` class method.

In [13]:
from dateutil import parser

from suncal.intervals import variables_reliability_target, variables_uncertainty_target, VariablesData

### Data in As-found and As-left format

If the data is given in as-found and as-left measurement values, use the `VariablesIntervalAssets` method to initialize the calculation.

If as-found equals as-left (i.e. no adjustments were made during any calibration), setting `use_all_deltas=True` will extract non-consecutive intervals from the as-found data, for example by including the interval between the first and third calibrations.

Here, we make up some random data, equally spaced in time. With all the actual intervals the same length, it is impossible to fit a curve to $\Delta y$ vs $\Delta t$. However, because no adjustments were made (yfound == yleft), `use_alldeltas` was set to extract other interval lengths and provide values that can be fit.

In [14]:
m = -.005  # Slope
s = .05    # Scatter
day = np.linspace(20, 40, num=10)   # Actual date of calibration
np.random.seed(342342)
yfound = np.random.normal(day*m, s)
data = VariablesData.from_assets(
    [{'enddates': day,
    'asfound': yfound}],
    use_alldeltas=True)
result = variables_reliability_target(data, -.5, .5, rel_conf=0.9)
result

### Interval: 48.46



![IMG0][]





[IMG0]: 

In [15]:
result = variables_uncertainty_target(data, .1)
result

### Interval: 70.38



![IMG0][]





[IMG0]: 

Information about the curve fit itself is available:

In [16]:
result.report.fit.summary()

Parameter,Value,Std. Uncertainty
a,-0.0074,0.0011


In [17]:
print(result.report.fit.b)     # Fit parameters as an array (length 1 for a line fit)
print(result.report.fit.cov)   # Covariance of fit parameters (1x1 for m=1 line fit)

[-0.00739283]
[[1.10768126e-06]]


## Data as $\Delta$t, $\Delta$y values

If the data is already formatted in terms of time-since-last-calibration and deviation-from-prior calibration, use `VariablesInterval(dt, dy)` to initialize the calculator.

This example approximates uses the data in Table 7-2 of NASA Handbook. Compare with results in Table 7-4 and 7-7.

In [18]:
dt = np.array([70, 86, 104, 135, 167, 173])
deltas = np.array([.1, .11, .251, .299, .403, .615])
data = VariablesData(dt, deltas, u0=.28, y0=10.03)
result1 = variables_reliability_target(data, 9, 11, rel_conf=.9, order=2)
result2 = variables_uncertainty_target(data, 0.5, order=2)
result1.interval, result2.interval  # Table 7-4 and 7-7

(140.09098164044073, 327.153339838551)

One-sided tolerances can be given by setting the other limit to None. Compare with results in NASA Table 7-5 and 7-6.

In [19]:
variables_reliability_target(data, None, 11, rel_conf=.9, order=2).interval   # Table 7-5

171.74799774746361

In [20]:
# Not sure this is a good example by NASA.. initial uncertainty is already below the target! But that's
# their example. NASA's table 7-2 shows the interval as 153, the point where the k=1.53 line
# crosses the lower limit. Note they changed the y0 value from the previous example.
data = VariablesData(dt, deltas, u0=.28, y0=9.03)
variables_reliability_target(data, 9, None, rel_conf=.9, order=2)  # Table 7-6

### Interval: 0.00



![IMG0][]





[IMG0]: 

Castrup's Paper has data in Table 1, but it does not match the later figures and screenshots.

In [21]:
# Test with data from Castrup Table 1
dates = [parser.parse('29-Mar-03'),
         parser.parse('11-Jul-03'),
         parser.parse('31-Dec-03'),
         parser.parse('15-May-04'),
         parser.parse('29-Oct-04'),
         parser.parse('23-Jan-05'),
         parser.parse('03-Apr-05')]

asfound = np.array([5.173, 5.123, 4.633, 4.915, 5.086, 4.913, 5.108])
asleft = np.array([5.073, 5.048, 4.993, 5.126, 5.024, 5.208, 5.451])
uncert = np.array([.2700, .2825, .2771, .2700, .2825, .2771, .2759])
datesord = np.array([d.toordinal() for d in dates])  # Ordinal day

data = VariablesData.from_assets([{
    'enddates': dates,
    'asfound': asfound,
    'asleft': asleft}],
                                u0=.25,
                                y0=0)
variables_reliability_target(data, -.810, .595, rel_conf=.8)

### Interval: 327.46



![IMG0][]





[IMG0]: 

In [22]:
variables_uncertainty_target(data, .4)


### Interval: 608.15



![IMG0][]





[IMG0]: 